In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import random
import time

In [ ]:
def calculate_pagecount(total_items):
    if int(total_items)%100 ==0:
        pages = int(total_items)//100
    else:
        pages = (int(total_items)//100)+1
    return pages

## Boardgame Classification

In [ ]:
ID = 306882

In [ ]:
api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}stats=1"

response = requests.get(api_adress)
response.status_code

In [ ]:
xml = response.text

In [ ]:
soup = BeautifulSoup(xml, 'xml')

In [ ]:
#print(soup.prettify())

In [ ]:
#soup.find_all('link')

In [ ]:
categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
categories

In [ ]:
mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
mechanics

In [ ]:
family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
family

In [ ]:
expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
expansions

In [ ]:
integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
integrations

In [ ]:
designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
designers

In [ ]:
publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
publishers 

## User Ratings

In [ ]:
api_adress = "https://www.boardgamegeek.com/xmlapi2/thing?id=306882&ratingcomments=1&page=1"

response = requests.get(api_adress)
response.status_code

In [ ]:
xml = response.text
soup = BeautifulSoup(xml, 'xml')

In [ ]:
print(soup.prettify())

In [ ]:
page = soup.find('comments').get('page')
page

In [ ]:
total_items = soup.find('comments').get('totalitems')
total_items

In [ ]:
pagecount = calculate_pagecount(total_items)
pagecount

In [ ]:
ratings = []
users = []
comments = []
for comment in soup.find_all('comment'):
    ratings.append(comment.get('rating'))
    users.append(comment.get('username'))
    comments.append(comment.get('value'))
    

In [ ]:
boardgame_id = [ID] * len(ratings)

In [ ]:
ratings = pd.DataFrame({
    'id':boardgame_id,
    'user':users,
    'rating':ratings,
    'comment':comments
})

In [ ]:
ratings['rating'].isna().sum()

To DO:
- Loop through pages
- ignore users without ratings
- User IDs?

## Loop over rating pages

In [ ]:
boardgames =pd.read_csv('../data/boardgames.csv', index_col='id')
boardgames['categories'] = None
boardgames['mechanics'] = None
boardgames['family'] = None
boardgames['expansions'] = None
boardgames['integrations'] = None
boardgames['designers'] = None
boardgames['publishers'] = None

ratings = pd.DataFrame()

for ID in [1692, 1881, 3044, 4493, 5668]:
    print(ID)
    user_ratings = []
    users = []
    comments = []
    page = 1
    print (f'page {page}')
    # request page 1 of game overview with user ratings and comments
    api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
    response = requests.get(api_adress)
    xml = response.text
    soup = BeautifulSoup(xml, 'xml')
    
    # get the number of pages of ratings
    total_items = soup.find('comments').get('totalitems')
    print(f'total items: {total_items}')
    pagecount = calculate_pagecount(total_items)
    print(f'pagecount: {pagecount}')

    # extend boardgames dataframe 
    categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
    boardgames.loc[ID,'categories'] = ', '.join(categories)
    mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
    boardgames.loc[ID,'mechanics'] = ', '.join(mechanics)
    family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
    boardgames.loc[ID,'family'] = ', '.join(family)
    expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
    boardgames.loc[ID,'expansions'] = ', '.join(expansions)
    integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
    boardgames.loc[ID,'integrations'] = ', '.join(integrations)
    designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
    boardgames.loc[ID,'designers'] = ', '.join(designers)
    publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
    boardgames.loc[ID,'publishers'] = ', '.join(publishers)
    
    # update number of voters
    boardgames.loc[ID, 'num_voters'] = total_items

    # if more than one page of comments send request for each page
    if pagecount > page:
        for comment in soup.find_all('comment'):
            user_ratings.append(comment.get('rating'))
            users.append(comment.get('username'))
            comments.append(comment.get('value'))        
        
        page = page + 1

        while page <= pagecount:
            print (f'page {page}')
            api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
            response = requests.get(api_adress)
            xml = response.text
            soup = BeautifulSoup(xml, 'xml')

            for comment in soup.find_all('comment'):
                user_ratings.append(comment.get('rating'))
                users.append(comment.get('username'))
                comments.append(comment.get('value'))

            page = page + 1
            time.sleep(3+random.random())
    # if only one page of comments, scrape from here
    else:
        for comment in soup.find_all('comment'):
            user_ratings.append(comment.get('rating'))
            users.append(comment.get('username'))
            comments.append(comment.get('value'))

    boardgame_id = [ID] * len(user_ratings)

    df = pd.DataFrame({
    'id':boardgame_id,
    'user':users,
    'rating':user_ratings,
    'comment':comments
    })
    
    ratings = ratings.append(df, ignore_index=True)
    
    ratings.to_csv('../data/ratings.csv', index=False)
    boardgames.to_csv('../data/boardgames_extend.csv', index=False)

In [ ]:
df

In [ ]:
len(user_ratings)

In [ ]:
len(boardgame_id)

In [ ]:
boardgames